In [32]:
# library
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [33]:
os.listdir('./ncaam-march-mania-2021')
stage1 = './ncaam-march-mania-2021/MDataFiles_Stage1/'
stage2 = './ncaam-march-mania-2021/MDataFiles_Stage2/'

In [34]:
season = pd.read_csv(stage2+'MRegularSeasonCompactResults.csv')
season

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0
...,...,...,...,...,...,...,...,...
170730,2021,132,1382,74,1433,65,N,0
170731,2021,132,1159,85,1259,72,H,0
170732,2021,132,1104,80,1261,79,N,0
170733,2021,132,1222,91,1153,54,N,0


In [35]:
tourney = pd.read_csv(stage2+'MNCAATourneyCompactResults.csv')
tourney

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0
...,...,...,...,...,...,...,...,...
2246,2019,146,1120,77,1246,71,N,1
2247,2019,146,1277,68,1181,67,N,0
2248,2019,152,1403,61,1277,51,N,0
2249,2019,152,1438,63,1120,62,N,0


In [36]:
ordinals = pd.read_csv(stage2+'MMasseyOrdinals.csv').rename(columns={'RankingDayNum':'DayNum'})
ordinals

,Season,DayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260
...,...,...,...,...,...
4339985,2021,133,WOB,1467,233
4339986,2021,133,WOB,1468,176
4339987,2021,133,WOB,1469,290
4339988,2021,133,WOB,1470,256


In [37]:
# Get the last available data from each system previous to the tournament
ordinals = ordinals.groupby(['SystemName','Season','TeamID']).last().reset_index()
ordinals

,SystemName,Season,TeamID,DayNum,OrdinalRank
0,7OT,2012,1102,133,146
1,7OT,2012,1103,133,80
2,7OT,2012,1104,133,35
3,7OT,2012,1105,133,338
4,7OT,2012,1106,133,300
...,...,...,...,...,...
349598,ZAM,2019,1462,133,70
349599,ZAM,2019,1463,133,87
349600,ZAM,2019,1464,133,242
349601,ZAM,2019,1465,133,198


In [38]:
# Add winner's ordinals
games = tourney.merge(ordinals,left_on=['Season','WTeamID'],right_on=['Season','TeamID'])

In [39]:
# Then add losser's ordinals
games = games.merge(ordinals,
left_on=['Season','LTeamID','SystemName'],right_on=['Season','TeamID','SystemName'],suffixes = ['W','L'])

In [40]:
## Add column with 1 if result is correct
games = games.drop(labels=['TeamIDW','TeamIDL'],axis=1)
games['prediction'] = (games.OrdinalRankW<games.OrdinalRankL).astype(int)
results_by_system = games.groupby('SystemName').agg({'prediction':('mean','count')})
results_by_system

prediction      
                 mean count
SystemName                 
7OT          0.701493   536
ACU          0.729508   244
ADE          0.686567   335
AP           0.618367   490
ARG          0.702174   460
...               ...   ...
WOB          0.703140  1051
WOL          0.706726  1115
WTE          0.640625    64
YAG          0.716418   201
ZAM          0.706468   201

[174 rows x 2 columns]

In [41]:
games['Wrating'] = 100-4*np.log(games['OrdinalRankW']+1)-games['OrdinalRankW']/22

In [42]:
games['Lrating'] = 100-4*np.log(games['OrdinalRankL']+1)-games['OrdinalRankL']/22

In [43]:
games['prob'] = 1/(1+10**((games['Lrating']-games['Wrating'])/15))

In [44]:
loss_results = games[games.Season>=2016].groupby('SystemName')['prob'].agg([('loss',lambda p: -np.mean(np.log(p))),('count','count')])

In [45]:
ref_system = 'POM'
ordinals['Rating']= 100-4*np.log(ordinals['OrdinalRank']+1)-ordinals['OrdinalRank']/22
ordinals = ordinals[ordinals.SystemName==ref_system]

In [46]:
# Get submission file
sub_df = pd.read_csv(stage2+'MSampleSubmissionStage2.csv')
sub_df['Season'] = sub_df['ID'].map(lambda x: int(x.split('_')[0]))
sub_df['Team1'] = sub_df['ID'].map(lambda x: int(x.split('_')[1]))
sub_df['Team2'] = sub_df['ID'].map(lambda x: int(x.split('_')[2]))
sub_df = sub_df.merge(ordinals[['Season','TeamID','Rating']], how='left', left_on = ['Season','Team1'], right_on = ['Season','TeamID'])
sub_df = sub_df.merge(ordinals[['Season','TeamID','Rating']], how='left', left_on = ['Season','Team2'], right_on = ['Season','TeamID'], suffixes=['W','L'])
sub_df['Pred'] = 1/(1+10**((sub_df['RatingL']-sub_df['RatingW'])/15))
sub_df[['ID', 'Pred']].to_csv('submission.csv', index=False, float_format='%.5g')